In [ ]:
!pip install tensorflow==2.12.0

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 7.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninsta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dir = '/content/drive/MyDrive/chest_xray'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Set parameters
img_height, img_width = 150, 150
batch_size = 32
num_classes = 2  # Normal and Pneumonia

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


Found 5236 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 33.4 MB/s eta 0:00:00


In [ ]:
import tenseal as ts

# Create a TenSEAL context
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192)
context.global_scale = 2**40

def encrypt_data(data):
    encrypted_data = []
    for img in data:
        # Flatten the image and encrypt
        img_flat = img.flatten()
        encrypted_img = ts.ckks_vector(context, img_flat.tolist())
        encrypted_data.append(encrypted_img)
    return encrypted_data

# Encrypt the training data
encrypted_train_data = encrypt_data(train_generator[0][0])  # Example for the first batch

The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_mod

In [ ]:
from tensorflow.keras import layers, models

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.AveragePooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.AveragePooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.AveragePooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
def decrypt_data(encrypted_data):
    decrypted_data = []
    for enc_img in encrypted_data:
        decrypted_img = enc_img.decrypt()
        # decrypted_data.append(np.array(decrypted_img))
        # **Reshape back to the original image shape**
        decrypted_data.append(np.array(decrypted_img).reshape((img_height, img_width, 3)))  # **Change made here**
    return np.array(decrypted_data)

In [ ]:
epochs = 5

# Custom training loop
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')

    for step in range(len(train_generator)):
        # Get the next batch of images and labels
        images, labels = train_generator.next()

        # **Reshape the labels to match the model's output shape**
        labels = np.expand_dims(labels, axis=-1)  # **Change made here**

        # Encrypt the images
        encrypted_images = encrypt_data(images)

        # Decrypt the images for the forward pass
        decrypted_images = decrypt_data(encrypted_images)

        # Using GradientTape for automatic differentiation
        with tf.GradientTape() as tape:
            # Forward pass
            predictions = model(decrypted_images, training=True)  # Use the model directly
            # Compute loss
            loss = tf.keras.losses.binary_crossentropy(labels, predictions)

        # Compute gradients
        gradients = tape.gradient(loss, model.trainable_variables)

        # Update model weights
        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        print(f'Step {step + 1}/{len(train_generator)}, Loss: {tf.reduce_mean(loss).numpy()}')
    print("ONE COMPLETED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

# Save the model
model.save('averagepoolingmodel.h5')


Streaming output truncated to the last 5000 lines.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need t

In [ ]:
# Data augmentation and preprocessing for the test dataset
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load the test dataset
test_generator = test_datagen.flow_from_directory(
    test_dir,  # Path to your test dataset
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  # Important to keep the order for evaluation
)


# Load the saved model
model = tf.keras.models.load_model('encrypedtraineddata.h5')
# model = tf.keras.models.load_model('chest2_xray_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Found 624 images belonging to 2 classes.
20/20 [==============================] - 16s 689ms/step - loss: 0.4976 - accuracy: 0.7885
Test Loss: 0.4976
Test Accuracy: 0.7885
